# Pseudocode zur Implementierung

In [20]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [21]:
# dieser Test-df wird von Christophs Programm erzeugt.
# wichtig!!! Bei der Besprechung müssen wir P_delta als NaN Value in den Datenframe einfügen
P_Phase_1 = pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P', 'P_delta'])
P_Phase_2 = pd.read_csv('power_today_minute_2.csv', names = ['timestamp','P', 'P_delta'])
P_Phase_3 = pd.read_csv('power_today_minute_3.csv', names = ['timestamp','P', 'P_delta'])

#Event dataframe anlegen zur späteren Auswertung der An/Abschaltevents
Event_df = pd.DataFrame(columns = ['timestamp','Phase','minuten_index',
                                   'Ladevorgang', 'Status', 'Ladeleistung',
                                   'minuten_index_Abschaltung','timestamp_abschalt'])
ladevorgang_df = pd.DataFrame(columns = ['timestamp_start','Phasen', 'Durchschnittliche Ladeleistung','Ladevorgang_ende'])

Leerlaufleistung_Phase = 120

## Funktion
-------

In [22]:
def add_p_delta(df):
    df_length = len(df)
    index_last_row = df_length - 1
    if(df_length > 1):
        p_new = df.iloc[index_last_row]['P']
        p_old = df.iloc[index_last_row-1]['P']
        p_delta = p_new - p_old
        return p_delta
    else:
        return 0

In [23]:
def detect_switch_event(rows, phase, Event_df):
    minuten_index = rows.index[-1]
    timestamp = rows.iloc[-1]['timestamp']
    ### Einschaltvorgang dedektieren ###
    # ist delta P relevant
    if(rows.iloc[-1]['P_delta'] > 400):
        # Prüfen ob Ladenvorgang beginnt/beendet
        # Wenn delta P positiv ---> event kreieren um später zu checken ob es ein Einschaltvorgang war
        Event_df.loc[len(Event_df)]= [timestamp, phase, minuten_index, 'ein', 'not checked', 0,0,0]
    
    #Check events: Überprüfen ob vorher eingetragenes event wirklich ein Einschaltvorgang ist
    event_df_length = len(Event_df)
    row_event = 0
    while row_event < event_df_length:
            #condition to check event
            minuten_index_event = Event_df.iloc[row_event]['minuten_index']
            #Check auf Einschaltvorgang
            if ((minuten_index_event+3 == minuten_index) & 
                (Event_df.iloc[row_event]['Ladevorgang'] == 'ein') &
                (Event_df.iloc[row_event]['Phase'] == phase)):
                Ladeleistung = rows.loc[minuten_index_event+3]['P'] - rows.loc[minuten_index_event-3]['P']            
                sum_delta_P = rows.loc[minuten_index_event:minuten_index_event+3]['P_delta'].sum()
                deviation = abs(Ladeleistung-sum_delta_P)/Ladeleistung 
                if ((deviation > 0.15) | (Ladeleistung < 1500)):
                    Event_df = Event_df.drop(row_event)
                    Event_df = Event_df.reset_index(drop=True)
                else:
                    Event_df.loc[row_event, ['Status', 'Ladeleistung']] = ['checked', Ladeleistung]
            event_df_length = len(Event_df)
            row_event += 1
         
    ### Ausschaltvorgang dedektieren ###
    #Checken ob aktuelle Leistung das Ende eines Einschaltvorgangs impliziert        
    if (('ein' in Event_df['Ladevorgang'].unique()) &
        ('checked' in Event_df['Status'].unique())):
        # create df with charging status on
        only_load_event_df = Event_df[(Event_df['Ladevorgang'] == 'ein') &
                                      (Event_df['minuten_index'] < minuten_index)]
        # Soll Leistung = Leistung die auf der Phase anliegen sollte,
        # wenn alle zuvor ladenden Autos noch an der Phase hängen würden
        Soll_Leistung = Leerlaufleistung_Phase + Event_df[(Event_df['Ladevorgang']=='ein') & 
                                                          (Event_df['minuten_index'] <= minuten_index) & 
                                                          (Event_df['minuten_index_Abschaltung'] == 0) & 
                                                          (Event_df['Phase'] == phase)]['Ladeleistung'].sum()
        #Aktuelle Leistung (neuer Messwert)
        Ist_Leistung = rows.iloc[-1]['P']
        #Residuale Leistung 
        P_residual = Soll_Leistung - Ist_Leistung
        #Überprüfen ob fehlende Leistung (P_residual) zu einem zuvor ladenden Auto passt
        only_load_event_df['deviation_P'] = only_load_event_df['Ladeleistung']. \
                                    apply(lambda ladeleistung_auto: abs(ladeleistung_auto-P_residual)/ladeleistung_auto)
        if ((only_load_event_df['deviation_P'] < 0.03).any()):
            # sort load events by size
            # und nehme das erste Event, was der aktuellen leistung am nächsten ist
            charg_off_ev_index = only_load_event_df['deviation_P'].sort_values().index[0]
            Event_df.loc[charg_off_ev_index,['minuten_index_Abschaltung']] = minuten_index
            Event_df.loc[charg_off_ev_index,['timestamp_abschalt']] = timestamp
            Event_df.loc[charg_off_ev_index, ['Ladevorgang']] = 'aus'
            
    return Event_df

In [24]:
def combine_charging_events(Event_df):
    # Zusammenfassen von mehrphasigen Ladevorängen
    for events in range(0,len(Event_df)):
        timestamp = Event_df.iloc[events]['timestamp']
        timestamp_abschaltung = Event_df.iloc[events]['timestamp_abschalt']
        Phase = []
        Ladeleistung = []
        same_events = Event_df.loc[Event_df['timestamp'] == Event_df.iloc[events]['timestamp']]
        event_count = len(same_events)
        for a in range (0,event_count):
            Phase.append(same_events.iloc[a]['Phase'])
            Ladeleistung.append(str(same_events.iloc[a]['Phase'])+': '+str(same_events.iloc[a]['Ladeleistung']))
        if (len(ladevorgang_df.loc[ladevorgang_df['timestamp_start'] == timestamp]) !=1):
            ladevorgang_df.loc[len(ladevorgang_df)]= [timestamp, Phase, Ladeleistung, timestamp_abschaltung]
            
    return ladevorgang_df

## Code Simulation auf dem Server
----
Hier kann der Code für eine Phase Simuliert werden

In [25]:
%%time

pseudocode_df_1 = pd.DataFrame([])
pseudocode_df_2 = pd.DataFrame([])
pseudocode_df_3 = pd.DataFrame([])
phasen_variable = 1


for row in range(0, len(P_Phase_1)):
    #Phase 1
    pseudocode_df_1 = pseudocode_df_1.append(P_Phase_1[row:row+1])
    
    # call the function to add P-delta
    pseudocode_df_1.iloc[-1, pseudocode_df_1.columns.get_loc('P_delta')] = add_p_delta(pseudocode_df_1[-2:])
    
    # Detect all switch events
    Event_df = detect_switch_event(pseudocode_df_1.iloc[-7:], 1, Event_df)
    
    # Phase 2
    pseudocode_df_2 = pseudocode_df_2.append(P_Phase_2[row:row+1])
    pseudocode_df_2.iloc[-1, pseudocode_df_2.columns.get_loc('P_delta')] = add_p_delta(pseudocode_df_2[-2:])
    Event_df = detect_switch_event(pseudocode_df_2.iloc[-7:], 2, Event_df)
    
    # Phase 3
    pseudocode_df_3 = pseudocode_df_3.append(P_Phase_3[row:row+1])
    pseudocode_df_3.iloc[-1, pseudocode_df_3.columns.get_loc('P_delta')] = add_p_delta(pseudocode_df_3[-2:])
    Event_df = detect_switch_event(pseudocode_df_3.iloc[-7:], 3, Event_df)
    
    

/Users/markus/anaconda/envs/ev_project/lib/python3.5/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 1min 1s, sys: 182 ms, total: 1min 1s
Wall time: 1min 1s


In [26]:
Event_df

,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505374e+09,2,575,aus,checked,3611,693,1.50538e+09
1,1.505377e+09,2,621,aus,checked,3528,931,1.5054e+09
2,1.505378e+09,3,640,aus,checked,3497,694,1.50538e+09
3,1.505386e+09,3,763,aus,checked,3496,947,1.5054e+09
4,1.505386e+09,3,772,aus,checked,3464,1102,1.50541e+09
5,1.505390e+09,3,832,aus,checked,3505,932,1.5054e+09
6,1.505390e+09,1,834,aus,checked,3590,1023,1.5054e+09
7,1.505390e+09,2,841,aus,checked,3617,1154,1.50541e+09
8,1.505392e+09,1,860,aus,checked,3630,1217,1.50541e+09
9,1.505392e+09,3,871,aus,checked,3439,906,1.50539e+09


In [27]:
combine_charging_events(Event_df)

,timestamp_start,Phasen,Durchschnittliche Ladeleistung,Ladevorgang_ende
0,1.505374e+09,[2],[2: 3611.0],1.505382e+09
1,1.505377e+09,[2],[2: 3528.0],1.505396e+09
2,1.505378e+09,[3],[3: 3497.0],1.505382e+09
3,1.505386e+09,[3],[3: 3496.0],1.505397e+09
4,1.505386e+09,[3],[3: 3464.0],1.505406e+09
5,1.505390e+09,[3],[3: 3505.0],1.505396e+09
6,1.505390e+09,[1],[1: 3590.0],1.505401e+09
7,1.505390e+09,[2],[2: 3617.0],1.505409e+09
8,1.505392e+09,[1],[1: 3630.0],1.505413e+09
9,1.505392e+09,[3],[3: 3439.0],1.505394e+09
